In [15]:
# Parameters
dataset = "imagenet" # CIFAR10, CIFAR100, SVHN
v_type = "mingd" # mingd, rand
root_path = "/home/r545zhan/aaa/False-Claims-Against-Model-Ownership-Resolution/defences/imagenet/DI/files"
params_path = "/home/r545zhan/aaa/False-Claims-Against-Model-Ownership-Resolution/defences/imagenet/DI/files"
split_index = 500


In [16]:
from IPython.display import display, Markdown
v_type2disp = dict(mingd="MinGD", rand="Blind Walk")
display(Markdown(f"# {dataset} - {v_type2disp[v_type]} results"))

# imagenet - MinGD results

In [17]:
import os, sys
sys.path.insert(0, params_path)

import argparse, time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from importlib import reload
from tqdm.auto import tqdm

import random

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [18]:
import seaborn as sns
import pandas as pd
import os

In [19]:
from scipy.stats import combine_pvalues, ttest_ind_from_stats, ttest_ind
from functools import reduce
from scipy.stats import hmean

In [20]:
root = os.path.join(root_path,dataset)

In [21]:
names = ["teacher","teacher_adv","fine-tune","independent","suspect","suspect_same_struct","suspect_same_data","suspect_adv","suspect_same_data_adv","suspect_same_struct_adv"]
trains = {}
tests = {}
for name in names:
    trains[name] = (torch.load(f"{root}/model_{name}/train_{v_type}_vulnerability_2.pt"))
    tests[name] = (torch.load(f"{root}/model_{name}/test_{v_type}_vulnerability_2.pt"))
mean_cifar = trains["teacher"].mean(dim = (0,1))
std_cifar = trains["teacher"].std(dim = (0,1))

In [22]:
trains["teacher"].shape

torch.Size([500, 10, 3])

In [23]:
for name in names:
    trains[name] = trains[name].sort(dim = 1)[0]
    tests[name] = tests[name].sort(dim = 1)[0]
for name in names:
    trains[name] = (trains[name]- mean_cifar)/std_cifar
    tests[name] = (tests[name]- mean_cifar)/std_cifar
f_num = 30
a_num=30

trains_n = {}
tests_n = {}
for name in names:
    if 'teacher' in name:
        trains_n[name] = trains[name].T.reshape(500,f_num)[:,:a_num]
    elif 'suspect_same_struct_adv'in name or "suspect_adv" in name:
        trains_n[name] = trains[name].T.reshape(500,f_num)[:,:a_num]
    elif "independent" in name:
        trains_n[name] = trains[name].T.reshape(1024,f_num)[:,:a_num]
    else:
        trains_n[name] = trains[name].T.reshape(500,f_num)[:,:a_num]
    tests_n[name] = tests[name].T.reshape(500,f_num)[:,:a_num]

/tmp/ipykernel_915982/2760032801.py:14: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/TensorShape.cpp:2981.)
  trains_n[name] = trains[name].T.reshape(500,f_num)[:,:a_num]


In [24]:
n_ex = split_index
train = torch.cat((trains_n["teacher"][:250], tests_n["teacher"][:250]), dim = 0)
y = torch.cat((torch.zeros(250), torch.ones(250)), dim = 0)
train_adv = torch.cat((trains_n["teacher_adv"][:250], tests_n["teacher_adv"][:250]), dim = 0)
rand=torch.randperm(y.shape[0])
train = train[rand]
train_adv = train_adv[rand]
y = y[rand]

In [25]:
model = nn.Sequential(nn.Linear(a_num,100),nn.ReLU(),nn.Linear(100,1),nn.Tanh())
criterion = nn.CrossEntropyLoss()
optimizer =torch.optim.SGD(model.parameters(), lr=0.1)
with tqdm(range(1000)) as pbar:
    for epoch in pbar:
        optimizer.zero_grad()
        inputs = train
        outputs = model(inputs)
        loss = -1 * ((2*y-1)*(outputs.squeeze(-1))).mean()
        loss.backward()
        optimizer.step()
        pbar.set_description('loss {}'.format(loss.item()))
model_adv = model
optimizer =torch.optim.SGD(model_adv.parameters(), lr=0.1)
with tqdm(range(1000)) as pbar:
    for epoch in pbar:
        optimizer.zero_grad()
        inputs = train_adv
        outputs = model(inputs)
        loss = -1 * ((2*y-1)*(outputs.squeeze(-1))).mean()
        loss.backward()
        optimizer.step()
        pbar.set_description('loss {}'.format(loss.item()))

loss -0.7259819507598877: 100%|██████████| 1000/1000 [00:01<00:00, 654.59it/s]


In [26]:
## Dataset Inference

In [27]:
def get_p(outputs_train, outputs_test):
    pred_test = outputs_test[:,0].detach().cpu().numpy()
    pred_train = outputs_train[:,0].detach().cpu().numpy()
    tval, pval = ttest_ind(pred_test, pred_train, alternative="greater", equal_var=False)
    if pval < 0:
        raise Exception(f"p-value={pval}")
    return pval

def get_p_values(num_ex, train, test, k):
    total = train.shape[0]
    sum_p = 0
    p_values = []
    positions_list = []
    for i in range(k):
        positions = torch.randperm(total)[:num_ex]
        p_val = get_p(train[positions], test[positions])
        positions_list.append(positions)
        p_values.append(p_val)
    return p_values

def get_fischer(num_ex, train, test, k):
    p_values = get_p_values(num_ex, train, test, k)
    return combine_pvalues(p_values, method="mudholkar_george")[1]

def get_max_p_value(num_ex, train, test, k):
    p_values = get_p_values(num_ex, train, test, k)
    return max(p_values)

In [28]:
outputs_tr = {}
outputs_te = {}
for name in names:
    if 'adv' not in name:
        model.eval()
        outputs_tr[name] = model(trains_n[name])
        outputs_te[name] = model(tests_n[name])
    else:
        model_adv.eval()
        outputs_tr[name] = model_adv(trains_n[name])
        outputs_te[name] = model_adv(tests_n[name])

In [29]:
import scipy.stats as stats
def print_inference(outputs_train, outputs_test):
    m1, m2 = outputs_test[:,0].mean(), outputs_train[:,0].mean()
    pval = get_p(outputs_train, outputs_test)
    print(f"p-value = {pval} \t| Mean difference = {m1-m2}\t|test_mu = {m1},train_mu = {m2}")
    return (m1-m2).item()

In [30]:
for name in names:
    outputs_tr[name], outputs_te[name] = outputs_tr[name][250:], outputs_te[name][250:]

In [31]:
mu={}
max_mu=0
for name in names:
    print(f"{name}")
    mu[name]=print_inference(outputs_tr[name], outputs_te[name])
    if mu[name]>=max_mu:
        max_mu = mu[name]

teacher
p-value = 3.466604886855402e-21 	| Mean difference = 0.57337486743927	|test_mu = 0.9404141306877136,train_mu = 0.3670392334461212
teacher_adv
p-value = 1.6869095097228255e-31 	| Mean difference = 0.858731210231781	|test_mu = 0.9404141306877136,train_mu = 0.08168291300535202
fine-tune
p-value = 1.527983895133034e-17 	| Mean difference = 0.5375710725784302	|test_mu = 0.9404374361038208,train_mu = 0.40286633372306824
independent
p-value = 5.317476409672829e-42 	| Mean difference = 0.5225328207015991	|test_mu = 0.9405975937843323,train_mu = 0.41806480288505554
suspect
p-value = 2.7175248647418982e-14 	| Mean difference = 0.4275725483894348	|test_mu = 0.9403398633003235,train_mu = 0.5127673149108887
suspect_same_struct
p-value = 1.9138758004102557e-08 	| Mean difference = 0.2649204730987549	|test_mu = 0.9403780102729797,train_mu = 0.6754575371742249
suspect_same_data
p-value = 2.2291505842308355e-24 	| Mean difference = 0.656856119632721	|test_mu = 0.9403539299964905,train_mu = 0.28

In [32]:
for name in names:
    print(name)
    print(mu[name]/max_mu)

teacher
0.6677000446793006
teacher_adv
1.0
fine-tune
0.6260062126230789
independent
0.6084940368716327
suspect
0.497911969769945
suspect_same_struct
0.3085022064439115
suspect_same_data
0.7649146925211071
suspect_adv
0.9999337827361598
suspect_same_data_adv
0.9999664749073011
suspect_same_struct_adv
0.9999170548948753
